<a href="https://colab.research.google.com/github/sittidetw/dads5001_midterm_project/blob/main/DADS5001_Midterm_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/sittidetw/dads5001_midterm_project.git

fatal: destination path 'dads5001_midterm_project' already exists and is not an empty directory.


In [2]:
pip install python-calamine

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import glob
import gc

# Data Cleaning

In [15]:
# @title HIV Testing by Key Populations
path = "/content/dads5001_midterm_project/rd_testing_population/*.xlsx"

files = glob.glob(path)
print("Total files:", len(files))
df_test_pop_combine = None
for file in files:
  print(file)

  # Load the Excel file
  df = pd.read_excel(file, engine='calamine', header=None, skiprows=2, skipfooter=1)
  file_name = Path(file).name
  file_period = file_name.split('_')
  period = file_period[2]

  df = df.transpose()
  df.iloc[:,0:3] = df.iloc[:,0:3].ffill()
  #df.insert(loc=0, column='header', value=df.index)
  #df

  # Define conditions
  conditions = [
      (df[0] == 'จังหวัด'),
      (df[0] == 'เขตสุขภาพ'),
      (df[0].str.contains('testing')) & (df[1].str.contains('Female')),
      (df[0].str.contains('testing')) & (df[1].str.contains('Male')),
      (df[0].str.contains('testing')) & (df[1].str.contains('All KP')),
      (df[0].str.contains('testing')) & (df[1].str.contains('Total')),
      (df[0].str.contains('Positive')) & (df[1].str.contains('Female')),
      (df[0].str.contains('Positive')) & (df[1].str.contains('Male')),
      (df[0].str.contains('Positive')) & (df[1].str.contains('All KP')),
      (df[0].str.contains('Positive')) & (df[1].str.contains('Total'))
  ]

  # Define the corresponding values for each condition
  values = ['province', 'health_region',
            'test_female', 'test_male', 'test_all_kp', 'test_total',
            'positive_female', 'positive_male', 'positive_all_kp', 'positive_total'
  ]

  # Create the new column
  df['header_temp'] = np.select(conditions, values, default='Unknown') # default is for cases that don't match any condition
  df['header'] = df['header_temp'].str.cat(df[2].str.replace(' ', '_').str.lower(), sep='_').fillna(df['header_temp'])
  df['header'] = df['header'].str.replace('total_total','grand_total')
  columns_to_drop = [0,1,2,'header_temp']
  df = df.drop(columns=columns_to_drop)
  df = df.transpose()
  df.columns = df.loc['header'].values
  df = df.drop(index='header')

  # Add Period
  df.insert(loc=0, column='quarter', value=period)
  df.insert(loc=0, column='q', value=period[-2:])
  df.insert(loc=0, column='year', value=period[:4])

  # Change data type in dataframe
  for col in df.iloc[0:,5:].columns: df[col] = df[col].astype('int32')
  for col in df.iloc[0:,0:5].columns: df[col] = df[col].astype('string')

  #df
  if (df_test_pop_combine is None):
    df_test_pop_combine = df
  else:
    df_test_pop_combine = pd.concat([df_test_pop_combine, df], ignore_index=True)

display(df_test_pop_combine.groupby(['year','quarter'])['positive_grand_total'].count())

# Transform accumulated data to total year summary
df_test_pop = df_test_pop_combine[df_test_pop_combine['q'] == 'Q4'].copy()
df_test_pop_temp1 = df_test_pop_combine[(df_test_pop_combine['q'] == 'Q1') & (df_test_pop_combine['year'].isin(df_test_pop['year'].unique()))].copy()

next_years_int = df_test_pop['year'].astype(int).unique() + 1
next_years_str = next_years_int.astype(str)
df_test_pop_temp2 = df_test_pop_combine[(df_test_pop_combine['q'] == 'Q1') & (df_test_pop_combine['year'].isin(next_years_str))].copy()

df_test_pop = df_test_pop.set_index(['year', 'province'])
df_test_pop_temp1 = df_test_pop_temp1.set_index(['year', 'province'])

df_test_pop_temp2['year'] = df_test_pop_temp2['year'].astype(int) - 1
df_test_pop_temp2['year'] = df_test_pop_temp2['year'].astype(str)
df_test_pop_temp2 = df_test_pop_temp2.set_index(['year', 'province'])

cols_to_cal = df_test_pop.columns[3:]
#df_test_pop[cols_to_cal] = (df_test_pop[cols_to_cal] - df_test_pop_temp1[cols_to_cal]) + df_test_pop_temp2[cols_to_cal]
df_test_pop[cols_to_cal] = df_test_pop[cols_to_cal].sub(df_test_pop_temp1[cols_to_cal], fill_value=0).add(df_test_pop_temp2[cols_to_cal], fill_value=0)

df_test_pop = df_test_pop.reset_index(drop=False)
df_test_pop = df_test_pop.drop(columns=['q','quarter'])
df_test_pop.iloc[:, 3:] = df_test_pop.iloc[:, 3:].clip(lower=0)

# Delete temp variables
del df_test_pop_temp1
del df_test_pop_temp2
gc.collect()

test_male = [col for col in df_test_pop.columns if col.startswith('test_male_') and 'all_kps' not in col and 'total' not in col]
test_female = [col for col in df_test_pop.columns if col.startswith('test_female_') and 'all_kps' not in col and 'total' not in col]
positive_male = [col for col in df_test_pop.columns if col.startswith('positive_male_') and 'all_kps' not in col and 'total' not in col]
positive_female = [col for col in df_test_pop.columns if col.startswith('positive_female_') and 'all_kps' not in col and 'total' not in col]

df_test_pop['test_male_all_kps'] = df_test_pop[test_male].sum(axis=1)
df_test_pop['test_female_all_kps'] = df_test_pop[test_female].sum(axis=1)
df_test_pop['positive_male_all_kps'] = df_test_pop[positive_male].sum(axis=1)
df_test_pop['positive_female_all_kps'] = df_test_pop[positive_female].sum(axis=1)

df_test_pop['test_all_kp_total'] = df_test_pop[['test_male_all_kps','test_female_all_kps']].sum(axis=1)
df_test_pop['positive_all_kp_total'] = df_test_pop[['positive_male_all_kps','positive_female_all_kps']].sum(axis=1)

# Add sum of others
added_col = ['test_male_others','test_female_others','test_all_others_total','positive_male_others','positive_female_others','positive_all_others_total']
col_name = ['test_male_all_kps','test_female_all_kps','test_all_kp_total','positive_male_all_kps','positive_female_all_kps','positive_all_kp_total']
col_ttl = ['test_male_total','test_female_total','test_grand_total','positive_male_total','positive_female_total','positive_grand_total']
indices = df_test_pop.columns.get_indexer(col_name)
for i in reversed(range(len(indices))):
    indice = indices[i]
    new_col = added_col[i]
    source_ttl = col_ttl[i]
    source_kp = col_name[i]
    df_test_pop.insert(loc=indice + 1, column=new_col, value=df_test_pop[source_ttl] - df_test_pop[source_kp])

display(df_test_pop)
df_test_pop.to_excel('test_pop.xlsx', index=False)

Total files: 7
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2025Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2026Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2024Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2025Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2023Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2023Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2024Q1_20260215.xlsx


year  quarter
2023  2023Q1     77
      2023Q4     77
2024  2024Q1     77
      2024Q4     77
2025  2025Q1     77
      2025Q4     77
2026  2026Q1     77
Name: positive_grand_total, dtype: int64

,year,province,health_region,test_male_fsw,test_male_migrant,test_male_msm,test_male_msw,test_male_partner_of_hiv,test_male_prisoner,test_male_pwid,...,positive_female_prisoner,positive_female_pwid,positive_female_tg,positive_female_tgsw,positive_female_all_kps,positive_female_others,positive_female_total,positive_all_kp_total,positive_all_others_total,positive_grand_total
0,2024,เชียงใหม่,1,0,2,19081,796,72,240,218,...,4,2,1,0,30,139,169,335,381,716
1,2024,ลำพูน,1,0,0,3174,0,8,3,0,...,1,0,0,0,4,26,30,29,64,93
2,2024,ลำปาง,1,0,0,2721,7,18,7,2,...,0,0,0,0,8,35,43,77,87,164
3,2024,แพร่,1,0,0,818,4,1,0,0,...,0,0,0,0,0,26,26,19,95,114
4,2024,น่าน,1,0,0,1539,2,18,185,2,...,0,0,0,0,1,13,14,41,43,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,2023,พัทลุง,12,0,1,1055,11,19,849,59,...,1,0,0,0,7,23,30,46,62,108
227,2023,ปัตตานี,12,0,0,423,1,5,1750,164,...,0,0,1,0,2,7,9,32,19,51
228,2023,ยะลา,12,0,1,189,1,13,1065,866,...,0,0,0,0,5,5,10,38,22,60
229,2023,นราธิวาส,12,0,0,255,3,5,0,375,...,0,0,0,0,3,12,15,32,43,75


In [16]:
df_test_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 57 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   year                            231 non-null    string
 1   province                        231 non-null    string
 2   health_region                   231 non-null    string
 3   test_male_fsw                   231 non-null    int32 
 4   test_male_migrant               231 non-null    int32 
 5   test_male_msm                   231 non-null    int32 
 6   test_male_msw                   231 non-null    int32 
 7   test_male_partner_of_hiv        231 non-null    int32 
 8   test_male_prisoner              231 non-null    int32 
 9   test_male_pwid                  231 non-null    int32 
 10  test_male_tg                    231 non-null    int32 
 11  test_male_tgsw                  231 non-null    int32 
 12  test_male_all_kps               231 non-null    in

In [17]:
# @title HIV Testing by Age
path = "/content/dads5001_midterm_project/rd_testing_agegr/*.xlsx"

files = glob.glob(path)
print("Total files:", len(files))
df_test_age_combine = None
for file in files:
  print(file)

  # Load the Excel file
  df = pd.read_excel(file, engine='calamine', header=None, skiprows=2, skipfooter=1)
  file_name = Path(file).name
  file_period = file_name.split('_')
  period = file_period[2]

  df = df.transpose()
  df.iloc[:,0:3] = df.iloc[:,0:3].ffill()
  #df.insert(loc=0, column='header', value=df.index)
  #df

  # Define conditions
  conditions = [
      (df[0] == 'จังหวัด'),
      (df[0] == 'เขตสุขภาพ'),
      (df[0].str.contains('testing')) & (df[1].str.contains('Female')),
      (df[0].str.contains('testing')) & (df[1].str.contains('Male')),
      (df[0].str.contains('testing')) & (df[1].str.contains('Total')),
      (df[0].str.contains('Positive')) & (df[1].str.contains('Female')),
      (df[0].str.contains('Positive')) & (df[1].str.contains('Male')),
      (df[0].str.contains('Positive')) & (df[1].str.contains('Total'))
  ]

  # Define the corresponding values for each condition
  values = ['province', 'health_region',
            'test_female', 'test_male', 'test_total',
            'positive_female', 'positive_male', 'positive_total'
  ]

  # Create the new column
  df['header_temp'] = np.select(conditions, values, default='Unknown') # default is for cases that don't match any condition
  df['header'] = df['header_temp'].str.cat(df[2].str.replace(' ', '_').str.lower(), sep='_').fillna(df['header_temp'])
  df['header'] = df['header'].str.replace('total_total','grand_total')
  columns_to_drop = [0,1,2,'header_temp']
  df = df.drop(columns=columns_to_drop)
  df = df.transpose()
  df.columns = df.loc['header'].values
  df = df.drop(index='header')

  # Add Period
  df.insert(loc=0, column='quarter', value=period)
  df.insert(loc=0, column='q', value=period[-2:])
  df.insert(loc=0, column='year', value=period[:4])

  # Change data type in dataframe
  for col in df.iloc[0:,5:].columns: df[col] = df[col].astype('int32')
  for col in df.iloc[0:,0:5].columns: df[col] = df[col].astype('string')

  #df
  if (df_test_age_combine is None):
    df_test_age_combine = df
  else:
    df_test_age_combine = pd.concat([df_test_age_combine, df], ignore_index=True)

display(df_test_age_combine.groupby(['year','quarter'])['positive_grand_total'].count())

# Transform accumulated data to total year summary
df_test_age = df_test_age_combine[df_test_age_combine['q'] == 'Q4'].copy()
df_test_age_temp1 = df_test_age_combine[(df_test_age_combine['q'] == 'Q1') & (df_test_age_combine['year'].isin(df_test_age['year'].unique()))].copy()

next_years_int = df_test_age['year'].astype(int).unique() + 1
next_years_str = next_years_int.astype(str)
df_test_age_temp2 = df_test_age_combine[(df_test_age_combine['q'] == 'Q1') & (df_test_age_combine['year'].isin(next_years_str))].copy()

df_test_age = df_test_age.set_index(['year', 'province'])
df_test_age_temp1 = df_test_age_temp1.set_index(['year', 'province'])

df_test_age_temp2['year'] = df_test_age_temp2['year'].astype(int) - 1
df_test_age_temp2['year'] = df_test_age_temp2['year'].astype(str)
df_test_age_temp2 = df_test_age_temp2.set_index(['year', 'province'])

cols_to_cal = df_test_age.columns[3:]
#df_test_age[cols_to_cal] = (df_test_age[cols_to_cal] - df_test_age_temp1[cols_to_cal]) + df_test_age_temp2[cols_to_cal]
df_test_age[cols_to_cal] = df_test_age[cols_to_cal].sub(df_test_age_temp1[cols_to_cal], fill_value=0).add(df_test_age_temp2[cols_to_cal], fill_value=0)

df_test_age = df_test_age.reset_index(drop=False)
df_test_age = df_test_age.drop(columns=['q','quarter'])
df_test_age.iloc[:, 3:] = df_test_age.iloc[:, 3:].clip(lower=0)

# Delete temp variables
del df_test_age_temp1
del df_test_age_temp2
gc.collect()

# ต้องเช็กก่อนว่า header ต้องใช้คำว่าอะไร และเปลี่ยนเงื่อนไข
test_male = [col for col in df_test_age.columns if col.startswith('test_male_') and 'total' not in col]
test_female = [col for col in df_test_age.columns if col.startswith('test_female_') and 'total' not in col]
positive_male = [col for col in df_test_age.columns if col.startswith('positive_male_') and 'total' not in col]
positive_female = [col for col in df_test_age.columns if col.startswith('positive_female_') and 'total' not in col]

df_test_age['test_male_total'] = df_test_age[test_male].sum(axis=1)
df_test_age['test_female_total'] = df_test_age[test_female].sum(axis=1)
df_test_age['positive_male_total'] = df_test_age[positive_male].sum(axis=1)
df_test_age['positive_female_total'] = df_test_age[positive_female].sum(axis=1)

df_test_age['test_grand_total'] = df_test_age[['test_male_total','test_female_total']].sum(axis=1)
df_test_age['positive_grand_total'] = df_test_age[['positive_male_total','positive_female_total']].sum(axis=1)

display(df_test_age)
df_test_age.to_excel('test_age.xlsx', index=False)

Total files: 7
/content/dads5001_midterm_project/rd_testing_agegr/Testing_AgeGr_2024Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_agegr/Testing_AgeGr_2023Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_agegr/Testing_AgeGr_2026Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_agegr/Testing_AgeGr_2025Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_agegr/Testing_AgeGr_2025Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_agegr/Testing_AgeGr_2023Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_agegr/Testing_AgeGr_2024Q4_20260215.xlsx


year  quarter
2023  2023Q1     77
      2023Q4     77
2024  2024Q1     77
      2024Q4     77
2025  2025Q1     77
      2025Q4     77
2026  2026Q1     77
Name: positive_grand_total, dtype: int64

,year,province,health_region,test_male_<1,test_male_1-4,test_male_5-9,test_male_10-14,test_male_15-19,test_male_20-24,test_male_25-49,...,positive_female_<1,positive_female_1-4,positive_female_5-9,positive_female_10-14,positive_female_15-19,positive_female_20-24,positive_female_25-49,positive_female_50+,positive_female_total,positive_grand_total
0,2025,เชียงใหม่,1,15,61,88,287,9452,10282,19999,...,0,0,0,0,10,24,104,48,186,691
1,2025,ลำพูน,1,0,2,0,72,1282,492,3158,...,0,0,0,0,2,3,26,4,35,117
2,2025,ลำปาง,1,0,3,5,11,2755,835,1799,...,0,0,0,0,1,5,18,17,41,156
3,2025,แพร่,1,0,0,1,17,1001,674,2441,...,0,0,0,0,0,1,14,6,21,127
4,2025,น่าน,1,0,10,6,27,1029,700,3125,...,0,0,0,0,1,2,11,5,19,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,2024,พัทลุง,12,1,12,30,186,1514,542,3700,...,0,0,0,0,0,3,14,13,30,113
227,2024,ปัตตานี,12,1,0,0,78,481,364,1605,...,0,0,0,0,0,0,8,3,11,71
228,2024,ยะลา,12,0,2,1,9,90,254,1563,...,0,0,0,0,0,0,17,3,20,70
229,2024,นราธิวาส,12,0,3,4,12,136,291,1265,...,0,0,0,0,2,3,8,8,21,85


In [18]:
df_test_age.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   year                   231 non-null    string
 1   province               231 non-null    string
 2   health_region          231 non-null    string
 3   test_male_<1           231 non-null    int32 
 4   test_male_1-4          231 non-null    int32 
 5   test_male_5-9          231 non-null    int32 
 6   test_male_10-14        231 non-null    int32 
 7   test_male_15-19        231 non-null    int32 
 8   test_male_20-24        231 non-null    int32 
 9   test_male_25-49        231 non-null    int32 
 10  test_male_50+          231 non-null    int32 
 11  test_male_total        231 non-null    int64 
 12  test_female_<1         231 non-null    int32 
 13  test_female_1-4        231 non-null    int32 
 14  test_female_5-9        231 non-null    int32 
 15  test_female_10-14      

In [23]:
# @title PrEP Cur Populations
path = "/content/dads5001_midterm_project/rd_prep_population/*.xlsx"

files = glob.glob(path)
print("Total files:", len(files))
df_prep_pop_combine = None
for file in files:
  print(file)

  # Load the Excel file
  df = pd.read_excel(file, engine='calamine', header=None, skiprows=2, skipfooter=1)
  file_name = Path(file).name
  file_period = file_name.split('_')
  period = file_period[3]

  df = df.transpose()
  df.iloc[:,0:3] = df.iloc[:,0:3].ffill()
  #df.insert(loc=0, column='header', value=df.index)
  #df

  # Define conditions
  conditions = [
      (df[0] == 'จังหวัด'),
      (df[0] == 'เขตสุขภาพ'),
      (df[0].str.lower().str.contains('prep')) & (df[1].str.contains('Female')),
      (df[0].str.lower().str.contains('prep')) & (df[1].str.contains('Male')),
      (df[0].str.lower().str.contains('prep')) & (df[1].str.contains('All KP')),
      (df[0].str.lower().str.contains('prep')) & (df[1].str.contains('Total')),
  ]

  # Define the corresponding values for each condition
  values = ['province', 'health_region',
            'prep_female', 'prep_male', 'prep_all_kp', 'prep_total'
  ]

  # Create the new column
  df['header_temp'] = np.select(conditions, values, default='Unknown') # default is for cases that don't match any condition
  df['header'] = df['header_temp'].str.cat(df[2].str.replace(' ', '_').str.lower(), sep='_').fillna(df['header_temp'])
  df['header'] = df['header'].str.replace('total_total','grand_total')
  columns_to_drop = [0,1,2,'header_temp']
  df = df.drop(columns=columns_to_drop)
  df = df.transpose()
  df.columns = df.loc['header'].values
  df = df.drop(index='header')

  # Add Period
  df.insert(loc=0, column='quarter', value=period)
  df.insert(loc=0, column='q', value=period[-2:])
  df.insert(loc=0, column='year', value=period[:4])

  # Change data type in dataframe
  for col in df.iloc[0:,5:].columns: df[col] = df[col].astype('int32')
  for col in df.iloc[0:,0:5].columns: df[col] = df[col].astype('string')

  #df
  if (df_prep_pop_combine is None):
    df_prep_pop_combine = df
  else:
    df_prep_pop_combine = pd.concat([df_prep_pop_combine, df], ignore_index=True)

display(df_prep_pop_combine.groupby(['year','quarter'])['prep_grand_total'].count())

# Transform accumulated data to total year summary
df_prep_pop = df_prep_pop_combine[df_prep_pop_combine['q'] == 'Q4'].copy()
df_prep_pop_temp1 = df_prep_pop_combine[(df_prep_pop_combine['q'] == 'Q1') & (df_prep_pop_combine['year'].isin(df_prep_pop['year'].unique()))].copy()

next_years_int = df_prep_pop['year'].astype(int).unique() + 1
next_years_str = next_years_int.astype(str)
df_prep_pop_temp2 = df_prep_pop_combine[(df_prep_pop_combine['q'] == 'Q1') & (df_prep_pop_combine['year'].isin(next_years_str))].copy()

df_prep_pop = df_prep_pop.set_index(['year', 'province'])
df_prep_pop_temp1 = df_prep_pop_temp1.set_index(['year', 'province'])

df_prep_pop_temp2['year'] = df_prep_pop_temp2['year'].astype(int) - 1
df_prep_pop_temp2['year'] = df_prep_pop_temp2['year'].astype(str)
df_prep_pop_temp2 = df_prep_pop_temp2.set_index(['year', 'province'])

cols_to_cal = df_prep_pop.columns[3:]
#df_prep_pop[cols_to_cal] = (df_prep_pop[cols_to_cal] - df_prep_pop_temp1[cols_to_cal]) + df_prep_pop_temp2[cols_to_cal]
df_prep_pop[cols_to_cal] = df_prep_pop[cols_to_cal].sub(df_prep_pop_temp1[cols_to_cal], fill_value=0).add(df_prep_pop_temp2[cols_to_cal], fill_value=0)

df_prep_pop = df_prep_pop.reset_index(drop=False)
df_prep_pop = df_prep_pop.drop(columns=['q','quarter'])
df_prep_pop.iloc[:, 3:] = df_prep_pop.iloc[:, 3:].clip(lower=0)
df_prep_pop = df_prep_pop.drop(columns=['prep_all_kp_total','prep_grand_total'])

# Delete temp variables
del df_prep_pop_temp1
del df_prep_pop_temp2
gc.collect()

prep_male = [col for col in df_prep_pop.columns if col.startswith('prep_male_') and 'all_kps' not in col and 'total' not in col]
prep_female = [col for col in df_prep_pop.columns if col.startswith('prep_female_') and 'all_kps' not in col and 'total' not in col]

df_prep_pop['prep_male_all_kps'] = df_prep_pop[prep_male].sum(axis=1)
df_prep_pop['prep_female_all_kps'] = df_prep_pop[prep_female].sum(axis=1)

# Add sum of others
added_col = ['prep_male_others','prep_female_others']
col_name = ['prep_male_all_kps','prep_female_all_kps']
col_ttl = ['prep_male_total','prep_female_total']
indices = df_prep_pop.columns.get_indexer(col_name)
for i in reversed(range(len(indices))):
    indice = indices[i]
    new_col = added_col[i]
    source_ttl = col_ttl[i]
    source_kp = col_name[i]
    df_prep_pop.insert(loc=indice + 1, column=new_col, value=df_prep_pop[source_ttl] - df_prep_pop[source_kp])
    df_prep_pop[new_col] = df_prep_pop[new_col].clip(lower=0)

df_prep_pop.iloc[:, 3:] = df_prep_pop.iloc[:, 3:].clip(lower=0)
df_prep_pop['prep_male_total'] = df_prep_pop[['prep_male_all_kps','prep_male_others']].sum(axis=1)
df_prep_pop['prep_female_total'] = df_prep_pop[['prep_female_all_kps','prep_female_others']].sum(axis=1)

# Melt to Long Format (Unpivot Columns)
df_prep_pop = df_prep_pop.melt(
    id_vars=['year', 'province', 'health_region'],
    var_name='temp_column',
    value_name='count'
)

split_data = df_prep_pop['temp_column'].str.split('_', expand=True)
df_prep_pop['sex'] = split_data[1]

df_prep_pop['category'] = split_data.iloc[:, 2:].fillna('').agg('_'.join, axis=1).str.strip('_')
df_prep_pop = df_prep_pop[['year', 'province', 'health_region', 'sex', 'category', 'count']]

# Pivot to Wide Format
df_prep_pop = df_prep_pop.pivot(
    index=['year', 'province', 'health_region', 'sex'],
    columns='category',
    values='count'
).reset_index()
df_prep_pop.columns.name = None

# Change Type
for col in df_prep_pop.iloc[0:,0:4].columns: df_prep_pop[col] = df_prep_pop[col].astype('string')
for col in df_prep_pop.iloc[0:,4:].columns: df_prep_pop[col] = df_prep_pop[col].astype('int64')

# Sort Columns
desired_order = ['year', 'province', 'health_region', 'sex', 'fsw', 'migrant', 'msm', 'msw', 'prisoner', 'pwid', 'tg', 'tgsw', 'all_kps', 'others', 'total']
actual_columns = [col for col in desired_order if col in df_prep_pop.columns]
df_prep_pop = df_prep_pop[actual_columns]

display(df_prep_pop.head())
display(df_prep_pop.info())
df_prep_pop.to_excel('prep_pop.xlsx', index=False)

Total files: 7
/content/dads5001_midterm_project/rd_prep_population/PrEP_Cur_Population_2025Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_population/PrEP_Cur_Population_2023Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_population/PrEP_Cur_Population_2026Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_population/PrEP_Cur_Population_2025Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_population/PrEP_Cur_Population_2024Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_population/PrEP_Cur_Population_2024Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_population/PrEP_Cur_Population_2023Q1_20260215.xlsx


year  quarter
2023  2023Q1     54
      2023Q4     67
2024  2024Q1     74
      2024Q4     74
2025  2025Q1     72
      2025Q4     76
2026  2026Q1     69
Name: prep_grand_total, dtype: int64

,year,province,health_region,sex,fsw,migrant,msm,msw,prisoner,pwid,tg,tgsw,all_kps,others,total
0,2023,กระบี่,11,female,0,0,0,0,0,0,0,0,0,4,4
1,2023,กระบี่,11,male,0,0,22,0,0,0,0,0,22,3,25
2,2023,กรุงเทพมหานคร,13,female,452,0,20,0,0,7,7,0,486,608,1094
3,2023,กรุงเทพมหานคร,13,male,0,0,14094,1088,1,91,1111,208,16593,379,16972
4,2023,กาญจนบุรี,5,female,2,0,0,0,0,0,0,0,2,5,7


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434 entries, 0 to 433
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year           434 non-null    string
 1   province       434 non-null    string
 2   health_region  434 non-null    string
 3   sex            434 non-null    string
 4   fsw            434 non-null    int64 
 5   migrant        434 non-null    int64 
 6   msm            434 non-null    int64 
 7   msw            434 non-null    int64 
 8   prisoner       434 non-null    int64 
 9   pwid           434 non-null    int64 
 10  tg             434 non-null    int64 
 11  tgsw           434 non-null    int64 
 12  all_kps        434 non-null    int64 
 13  others         434 non-null    int64 
 14  total          434 non-null    int64 
dtypes: int64(11), string(4)
memory usage: 51.0 KB


None

In [20]:
df_prep_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   year                   217 non-null    string 
 1   province               217 non-null    string 
 2   health_region          217 non-null    string 
 3   prep_male_fsw          217 non-null    float64
 4   prep_male_migrant      217 non-null    float64
 5   prep_male_msm          217 non-null    float64
 6   prep_male_msw          217 non-null    float64
 7   prep_male_prisoner     217 non-null    float64
 8   prep_male_pwid         217 non-null    float64
 9   prep_male_tg           217 non-null    float64
 10  prep_male_tgsw         217 non-null    float64
 11  prep_male_all_kps      217 non-null    float64
 12  prep_male_others       217 non-null    float64
 13  prep_male_total        217 non-null    float64
 14  prep_female_fsw        217 non-null    float64
 15  prep_f

In [21]:
# @title PrEP Cur by Age
path = "/content/dads5001_midterm_project/rd_prep_agegr/*.xlsx"

files = glob.glob(path)
print("Total files:", len(files))
df_prep_age_combine = None
for file in files:
  print(file)

  # Load the Excel file
  df = pd.read_excel(file, engine='calamine', header=None, skiprows=2, skipfooter=1)
  file_name = Path(file).name
  file_period = file_name.split('_')
  period = file_period[3]

  df = df.transpose()
  df.iloc[:,0:3] = df.iloc[:,0:3].ffill()
  #df.insert(loc=0, column='header', value=df.index)
  #df

  # Define conditions
  conditions = [
      (df[0] == 'จังหวัด'),
      (df[0] == 'เขตสุขภาพ'),
      (df[0].str.lower().str.contains('prep')) & (df[1].str.contains('Female')),
      (df[0].str.lower().str.contains('prep')) & (df[1].str.contains('Male')),
      (df[0].str.lower().str.contains('prep')) & (df[1].str.contains('Total')),
  ]

  # Define the corresponding values for each condition
  values = ['province', 'health_region',
            'prep_female', 'prep_male', 'prep_total'
  ]

  # Create the new column
  df['header_temp'] = np.select(conditions, values, default='Unknown') # default is for cases that don't match any condition
  df['header'] = df['header_temp'].str.cat(df[2].str.replace(' ', '_').str.lower(), sep='_').fillna(df['header_temp'])
  df['header'] = df['header'].str.replace('total_total','grand_total')
  columns_to_drop = [0,1,2,'header_temp']
  df = df.drop(columns=columns_to_drop)
  df = df.transpose()
  df.columns = df.loc['header'].values
  df = df.drop(index='header')

  # Add Period
  df.insert(loc=0, column='quarter', value=period)
  df.insert(loc=0, column='q', value=period[-2:])
  df.insert(loc=0, column='year', value=period[:4])

  # Change data type in dataframe
  for col in df.iloc[0:,5:].columns: df[col] = df[col].astype('int32')
  for col in df.iloc[0:,0:5].columns: df[col] = df[col].astype('string')

  #df
  if (df_prep_age_combine is None):
    df_prep_age_combine = df
  else:
    df_prep_age_combine = pd.concat([df_prep_age_combine, df], ignore_index=True)

display(df_prep_age_combine.groupby(['year','quarter'])['prep_grand_total'].count())

# Transform accumulated data to total year summary
df_prep_age = df_prep_age_combine[df_prep_age_combine['q'] == 'Q4'].copy()
df_prep_age_temp1 = df_prep_age_combine[(df_prep_age_combine['q'] == 'Q1') & (df_prep_age_combine['year'].isin(df_prep_age['year'].unique()))].copy()

next_years_int = df_prep_age['year'].astype(int).unique() + 1
next_years_str = next_years_int.astype(str)
df_prep_age_temp2 = df_prep_age_combine[(df_prep_age_combine['q'] == 'Q1') & (df_prep_age_combine['year'].isin(next_years_str))].copy()

df_prep_age = df_prep_age.set_index(['year', 'province'])
df_prep_age_temp1 = df_prep_age_temp1.set_index(['year', 'province'])

df_prep_age_temp2['year'] = df_prep_age_temp2['year'].astype(int) - 1
df_prep_age_temp2['year'] = df_prep_age_temp2['year'].astype(str)
df_prep_age_temp2 = df_prep_age_temp2.set_index(['year', 'province'])

cols_to_cal = df_prep_age.columns[3:]
#df_prep_age[cols_to_cal] = (df_prep_age[cols_to_cal] - df_prep_age_temp1[cols_to_cal]) + df_prep_age_temp2[cols_to_cal]
df_prep_age[cols_to_cal] = df_prep_age[cols_to_cal].sub(df_prep_age_temp1[cols_to_cal], fill_value=0).add(df_prep_age_temp2[cols_to_cal], fill_value=0)

df_prep_age = df_prep_age.reset_index(drop=False)
df_prep_age = df_prep_age.drop(columns=['q','quarter'])
df_prep_age.iloc[:, 3:] = df_prep_age.iloc[:, 3:].clip(lower=0)

# Delete temp variables
del df_prep_age_temp1
del df_prep_age_temp2
gc.collect()

prep_male = [col for col in df_prep_age.columns if col.startswith('prep_male_') and 'total' not in col]
prep_female = [col for col in df_prep_age.columns if col.startswith('prep_female_') and 'total' not in col]

df_prep_age['prep_male_total'] = df_prep_age[prep_male].sum(axis=1)
df_prep_age['prep_female_total'] = df_prep_age[prep_female].sum(axis=1)

df_prep_age['prep_grand_total'] = df_prep_age[['prep_male_total','prep_female_total']].sum(axis=1)

display(df_prep_age)
df_prep_age.to_excel('prep_age.xlsx', index=False)

Total files: 7
/content/dads5001_midterm_project/rd_prep_agegr/PrEP_Cur_AgeGr_2026Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_agegr/PrEP_Cur_AgeGr_2025Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_agegr/PrEP_Cur_AgeGr_2025Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_agegr/PrEP_Cur_AgeGr_2023Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_agegr/PrEP_Cur_AgeGr_2023Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_agegr/PrEP_Cur_AgeGr_2024Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_agegr/PrEP_Cur_AgeGr_2024Q1_20260215.xlsx


year  quarter
2023  2023Q1     54
      2023Q4     67
2024  2024Q1     74
      2024Q4     74
2025  2025Q1     72
      2025Q4     76
2026  2026Q1     69
Name: prep_grand_total, dtype: int64

,year,province,health_region,prep_male_<1,prep_male_1-4,prep_male_5-9,prep_male_10-14,prep_male_15-19,prep_male_20-24,prep_male_25-49,...,prep_female_<1,prep_female_1-4,prep_female_5-9,prep_female_10-14,prep_female_15-19,prep_female_20-24,prep_female_25-49,prep_female_50+,prep_female_total,prep_grand_total
0,2025,เชียงใหม่,1,0.0,0.0,0.0,0.0,76.0,418.0,1601.0,...,0.0,0.0,0.0,0.0,5.0,16.0,136.0,12.0,169.0,2350.0
1,2025,ลำพูน,1,0.0,0.0,0.0,0.0,0.0,0.0,21.0,...,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,5.0,27.0
2,2025,ลำปาง,1,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,5.0
3,2025,แพร่,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,3.0,4.0
4,2025,น่าน,1,0.0,0.0,0.0,0.0,2.0,6.0,43.0,...,0.0,0.0,0.0,0.0,1.0,0.0,5.0,2.0,8.0,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,2024,พัทลุง,12,0.0,0.0,0.0,0.0,5.0,1.0,17.0,...,0.0,0.0,0.0,0.0,6.0,1.0,9.0,1.0,17.0,41.0
213,2024,ปัตตานี,12,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
214,2024,ยะลา,12,0.0,0.0,0.0,0.0,0.0,1.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
215,2024,นราธิวาส,12,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.0,6.0,7.0


In [22]:
df_prep_age.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               217 non-null    string 
 1   province           217 non-null    string 
 2   health_region      217 non-null    string 
 3   prep_male_<1       217 non-null    float64
 4   prep_male_1-4      217 non-null    float64
 5   prep_male_5-9      217 non-null    float64
 6   prep_male_10-14    217 non-null    float64
 7   prep_male_15-19    217 non-null    float64
 8   prep_male_20-24    217 non-null    float64
 9   prep_male_25-49    217 non-null    float64
 10  prep_male_50+      217 non-null    float64
 11  prep_male_total    217 non-null    float64
 12  prep_female_<1     217 non-null    float64
 13  prep_female_1-4    217 non-null    float64
 14  prep_female_5-9    217 non-null    float64
 15  prep_female_10-14  217 non-null    float64
 16  prep_female_15-19  217 non

# Test

In [28]:
# @title PrEP Cur by Age
path = "/content/dads5001_midterm_project/rd_prep_agegr/*.xlsx"

files = glob.glob(path)
print("Total files:", len(files))
df_prep_age_combine = None
for file in files:
  print(file)

  # Load the Excel file
  df = pd.read_excel(file, engine='calamine', header=None, skiprows=2, skipfooter=1)
  file_name = Path(file).name
  file_period = file_name.split('_')
  period = file_period[3]

  df = df.transpose()
  df.iloc[:,0:3] = df.iloc[:,0:3].ffill()
  #df.insert(loc=0, column='header', value=df.index)
  #df

  # Define conditions
  conditions = [
      (df[0] == 'จังหวัด'),
      (df[0] == 'เขตสุขภาพ'),
      (df[0].str.lower().str.contains('prep')) & (df[1].str.contains('Female')),
      (df[0].str.lower().str.contains('prep')) & (df[1].str.contains('Male')),
      (df[0].str.lower().str.contains('prep')) & (df[1].str.contains('Total')),
  ]

  # Define the corresponding values for each condition
  values = ['province', 'health_region',
            'prep_female', 'prep_male', 'prep_total'
  ]

  # Create the new column
  df['header_temp'] = np.select(conditions, values, default='Unknown') # default is for cases that don't match any condition
  df['header'] = df['header_temp'].str.cat(df[2].str.replace(' ', '_').str.lower(), sep='_').fillna(df['header_temp'])
  df['header'] = df['header'].str.replace('total_total','grand_total')
  columns_to_drop = [0,1,2,'header_temp']
  df = df.drop(columns=columns_to_drop)
  df = df.transpose()
  df.columns = df.loc['header'].values
  df = df.drop(index='header')

  # Add Period
  df.insert(loc=0, column='quarter', value=period)
  df.insert(loc=0, column='q', value=period[-2:])
  df.insert(loc=0, column='year', value=period[:4])

  # Change data type in dataframe
  for col in df.iloc[0:,5:].columns: df[col] = df[col].astype('int32')
  for col in df.iloc[0:,0:5].columns: df[col] = df[col].astype('string')

  #df
  if (df_prep_age_combine is None):
    df_prep_age_combine = df
  else:
    df_prep_age_combine = pd.concat([df_prep_age_combine, df], ignore_index=True)

display(df_prep_age_combine.groupby(['year','quarter'])['prep_grand_total'].count())

# Transform accumulated data to total year summary
df_prep_age = df_prep_age_combine[df_prep_age_combine['q'] == 'Q4'].copy()
df_prep_age_temp1 = df_prep_age_combine[(df_prep_age_combine['q'] == 'Q1') & (df_prep_age_combine['year'].isin(df_prep_age['year'].unique()))].copy()

next_years_int = df_prep_age['year'].astype(int).unique() + 1
next_years_str = next_years_int.astype(str)
df_prep_age_temp2 = df_prep_age_combine[(df_prep_age_combine['q'] == 'Q1') & (df_prep_age_combine['year'].isin(next_years_str))].copy()

df_prep_age = df_prep_age.set_index(['year', 'province'])
df_prep_age_temp1 = df_prep_age_temp1.set_index(['year', 'province'])

df_prep_age_temp2['year'] = df_prep_age_temp2['year'].astype(int) - 1
df_prep_age_temp2['year'] = df_prep_age_temp2['year'].astype(str)
df_prep_age_temp2 = df_prep_age_temp2.set_index(['year', 'province'])

cols_to_cal = df_prep_age.columns[3:]
#df_prep_age[cols_to_cal] = (df_prep_age[cols_to_cal] - df_prep_age_temp1[cols_to_cal]) + df_prep_age_temp2[cols_to_cal]
df_prep_age[cols_to_cal] = df_prep_age[cols_to_cal].sub(df_prep_age_temp1[cols_to_cal], fill_value=0).add(df_prep_age_temp2[cols_to_cal], fill_value=0)

df_prep_age = df_prep_age.reset_index(drop=False)
df_prep_age = df_prep_age.drop(columns=['q','quarter'])
df_prep_age.iloc[:, 3:] = df_prep_age.iloc[:, 3:].clip(lower=0)

# Delete temp variables
del df_prep_age_temp1
del df_prep_age_temp2
gc.collect()

prep_male = [col for col in df_prep_age.columns if col.startswith('prep_male_') and 'total' not in col]
prep_female = [col for col in df_prep_age.columns if col.startswith('prep_female_') and 'total' not in col]

df_prep_age['prep_male_total'] = df_prep_age[prep_male].sum(axis=1)
df_prep_age['prep_female_total'] = df_prep_age[prep_female].sum(axis=1)

df_prep_age = df_prep_age.drop(columns='prep_grand_total')

# Melt to Long Format (Unpivot Columns)
df_prep_age = df_prep_age.melt(
    id_vars=['year', 'province', 'health_region'],
    var_name='temp_column',
    value_name='count'
)

split_data = df_prep_age['temp_column'].str.split('_', expand=True)
df_prep_age['sex'] = split_data[1]

df_prep_age['category'] = split_data.iloc[:, 2:].fillna('').agg('_'.join, axis=1).str.strip('_')
df_prep_age = df_prep_age[['year', 'province', 'health_region', 'sex', 'category', 'count']]

# Pivot to Wide Format
df_prep_age = df_prep_age.pivot(
    index=['year', 'province', 'health_region', 'sex'],
    columns='category',
    values='count'
).reset_index()
df_prep_age.columns.name = None

# Change Type
for col in df_prep_age.iloc[0:,0:4].columns: df_prep_age[col] = df_prep_age[col].astype('string')
for col in df_prep_age.iloc[0:,4:].columns: df_prep_age[col] = df_prep_age[col].astype('int64')

# Sort Columns
desired_order = ['year', 'province', 'health_region', 'sex', '<1', '1-4', '5-9', '10-14', '15-19', '20-24', '25-49', '50+', 'total']
actual_columns = [col for col in desired_order if col in df_prep_age.columns]
df_prep_age = df_prep_age[actual_columns]

display(df_prep_age.head())
display(df_prep_age.info())
df_prep_age.to_excel('prep_age.xlsx', index=False)

Total files: 7
/content/dads5001_midterm_project/rd_prep_agegr/PrEP_Cur_AgeGr_2026Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_agegr/PrEP_Cur_AgeGr_2025Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_agegr/PrEP_Cur_AgeGr_2025Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_agegr/PrEP_Cur_AgeGr_2023Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_agegr/PrEP_Cur_AgeGr_2023Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_agegr/PrEP_Cur_AgeGr_2024Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_prep_agegr/PrEP_Cur_AgeGr_2024Q1_20260215.xlsx


year  quarter
2023  2023Q1     54
      2023Q4     67
2024  2024Q1     74
      2024Q4     74
2025  2025Q1     72
      2025Q4     76
2026  2026Q1     69
Name: prep_grand_total, dtype: int64

,year,province,health_region,sex,<1,1-4,5-9,10-14,15-19,20-24,25-49,50+,total
0,2023,กระบี่,11,female,0,0,0,0,0,0,4,0,4
1,2023,กระบี่,11,male,0,0,0,0,0,0,25,0,25
2,2023,กรุงเทพมหานคร,13,female,0,0,0,3,69,252,737,33,1094
3,2023,กรุงเทพมหานคร,13,male,21,0,0,3,461,2951,13086,450,16972
4,2023,กาญจนบุรี,5,female,0,0,0,0,0,0,7,0,7


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434 entries, 0 to 433
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   year           434 non-null    string
 1   province       434 non-null    string
 2   health_region  434 non-null    string
 3   sex            434 non-null    string
 4   <1             434 non-null    int64 
 5   1-4            434 non-null    int64 
 6   5-9            434 non-null    int64 
 7   10-14          434 non-null    int64 
 8   15-19          434 non-null    int64 
 9   20-24          434 non-null    int64 
 10  25-49          434 non-null    int64 
 11  50+            434 non-null    int64 
 12  total          434 non-null    int64 
dtypes: int64(9), string(4)
memory usage: 44.2 KB


None